# Derive models of spacecraft with star tracker

Do all imports.

In [1]:
import sympy as sym
import numpy as np
import ae353_spacecraft_design as design

from scipy import linalg
from scipy import signal
import random

## Create spacecraft

Create a visualizer to help with placement of reaction wheels.

In [2]:
# Create the visualizer
vis = design.create_visualizer()

# Show the visualizer in this notebook
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Specify the location of each reaction wheel in terms of its right ascension $\alpha$ and declination $\delta$:

In [3]:
wheels = [
    {'alpha': np.pi/3, 'delta': 0.},
    {'alpha': 3*np.pi/3, 'delta': 0},
    {'alpha': -np.pi/3, 'delta': 0.},
    {'alpha': 0, 'delta': 3*np.pi/2},
]

Show wheels in the visualizer. You will be warned if any wheel obscures the star tracker (i.e., the "scope") or if any two wheels are too close together.

In [4]:
design.show_wheels(vis, wheels)

Create a model of the spacecraft in URDF format. This will **overwrite** the file `spacecraft.urdf` in the `urdf` directory.

In [5]:
design.create_spacecraft(wheels)

## Create dynamic model

Specify the physical parameters:

In [6]:
# Mass and MOI of base
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

# Mass and MOI of each wheel
mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 2.2

Derive the equations of motion:

In [7]:
# Define yaw, pitch, roll angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Compute resultant torques
T1 = - tau_1 * sym.Matrix(wheels[0]['xyz'])
T2 = - tau_2 * sym.Matrix(wheels[1]['xyz'])
T3 = - tau_3 * sym.Matrix(wheels[2]['xyz'])
T4 = - tau_4 * sym.Matrix(wheels[3]['xyz'])
T = sym.nsimplify(T1 + T2 + T3 + T4)

# Define MOI of spacecraft and wheels together
#
#  FIXME: For now, assume that each RW is a point mass. Later,
#         somebody should do this properly.
#
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# Define rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

# Define the transformation from angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Define euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# Define equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \\ \dot{w_x} \\ \dot{w_y} \\ \dot{w_z} \end{bmatrix} = f\left(\psi, \theta, \phi, w_x, w_y, w_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [8]:
f

Matrix([
[                                                                                                                  (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                   -55*tau_1/1468 + 55*tau_2/734 - 55*tau_3/1468 + 404133443718627*tau_4/29360000000000000000000000000000 - 75*w_y*w_z/367],
[-190525588832577*tau_1/2936000000000000 - 134711147906209*tau_2/14680000000000000000000000000000 + 190525588832577*tau_3/2936000000000000 + 75*w_x*w_z/367],
[                                                                                                                                              55*tau_4/884]])

## Create sensor model

Symbolic variables for right ascension $\alpha$ and declination $\delta$ of each star:

In [9]:
alpha, delta = sym.symbols('alpha, delta')

Specify the physical parameters:

In [10]:
# Scope radius
r = 0.8 / 2.1

Derive the sensor model:

In [11]:
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rz * Ry * Rx

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

The sensor model has this form for each star:

$$\zeta = g(\psi, \theta, \phi, \alpha, \delta)$$

Here is the function $g$:

In [12]:
g

Matrix([
[21*(sin(delta)*sin(phi)*cos(theta) + sin(phi)*sin(theta)*cos(delta)*cos(alpha - psi) + sin(alpha - psi)*cos(delta)*cos(phi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))],
[21*(sin(delta)*cos(phi)*cos(theta) - sin(phi)*sin(alpha - psi)*cos(delta) + sin(theta)*cos(delta)*cos(phi)*cos(alpha - psi))/(8*(-sin(delta)*sin(theta) + cos(delta)*cos(theta)*cos(alpha - psi)))]])

## Finding A & B for F

In [13]:
f

Matrix([
[                                                                                                                  (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                   -55*tau_1/1468 + 55*tau_2/734 - 55*tau_3/1468 + 404133443718627*tau_4/29360000000000000000000000000000 - 75*w_y*w_z/367],
[-190525588832577*tau_1/2936000000000000 - 134711147906209*tau_2/14680000000000000000000000000000 + 190525588832577*tau_3/2936000000000000 + 75*w_x*w_z/367],
[                                                                                                                                              55*tau_4/884]])

#### find equilibrium values for f

In [14]:
tau1e = 0
tau2e = 0
tau3e = 0
tau4e = 0
phie = 0
thetae = 0
psie = 0
wxe = 0
wye = 0
wze = 0

# Define yaw, pitch, roll angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

A_num = sym.lambdify((psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4), f.jacobian([psi, theta, phi, w_x, w_y, w_z]))
B_num = sym.lambdify((psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4), f.jacobian([tau_1, tau_2, tau_3, tau_4]))

A = A_num(psie, thetae, phie, wxe, wye, wze, tau1e, tau2e, tau3e, tau4e).astype(float)
B = B_num(psie, thetae, phie, wxe, wye, wze, tau1e, tau2e, tau3e, tau4e).astype(float)

In [15]:
A, B

(array([[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1., -0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -0., -0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-3.74659401e-02,  7.49318801e-02, -3.74659401e-02,
          1.37647631e-17],
        [-6.48929117e-02, -9.17650871e-18,  6.48929117e-02,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.22171946e-02]]))

In [16]:
nn = A.shape[0]
W = B
for i in range(1, nn):
    col = np.linalg.matrix_power(A, i)@B
    W = np.block([W, col])

In [17]:
np.linalg.matrix_rank(W), nn

(6, 6)

In [18]:
w = np.block([B, A@B, A@A@B, A@A@A@B, A@A@A@A@B, A@A@A@A@A@B])
np.linalg.matrix_rank(w), nn

(6, 6)

## A and B Matrices

In [19]:
print(A)
print(B)

[[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1. -0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -0. -0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.74659401e-02  7.49318801e-02 -3.74659401e-02  1.37647631e-17]
 [-6.48929117e-02 -9.17650871e-18  6.48929117e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.22171946e-02]]


## LQR for Controller

#### LQR function definition

In [20]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @ B.T @ P
    return K

#### Q and R Matrices

In [21]:
Q = np.diag([1, 1, 1, 1, 1, 1])
R = np.eye(4)

In [22]:
Q, R

(array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]))

#### K Matrix using LQR

In [23]:
K = lqr(A, B, Q, R)

In [24]:
print(K)

[[ 7.82718250e-16 -7.07106781e-01 -4.08248290e-01 -1.94906114e+00
  -3.37587293e+00  2.65757997e-15]
 [-7.68735572e-16  9.70141776e-16  8.16496581e-01  3.89812228e+00
   7.43027507e-16 -3.21738452e-15]
 [-1.39826783e-17  7.07106781e-01 -4.08248290e-01 -1.94906114e+00
   3.37587293e+00  5.59804552e-16]
 [ 1.00000000e+00 -1.00273328e-16 -2.59770982e-16 -1.95537478e-15
  -1.00563912e-15  5.75720892e+00]]


## Observer Design, use G

#### what we need: L, C

#### for every star, have [[y],[z]], which is input for g

In [25]:
alphas = [-.1, 0, 0.1, 0, -0.1, 0, 0.1]
deltas = [-0.15, -0.15, -0.15, 0, .15, .15, .15]
alphas, deltas

([-0.1, 0, 0.1, 0, -0.1, 0, 0.1], [-0.15, -0.15, -0.15, 0, 0.15, 0.15, 0.15])

#### modified g matrix to find C

In [26]:
gs = sym.zeros(14, 1)
for i in range(0, 7):
    gs[i*2] = g.subs(alpha, alphas[i]).subs(delta, deltas[i])[0]
    gs[i*2 + 1] = g.subs(alpha, alphas[i]).subs(delta, deltas[i])[1]

In [27]:
gs

Matrix([
[ 21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi + 0.1) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi + 0.1)*cos(phi))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1)))],
[ 21*(0.988771077936042*sin(phi)*sin(psi + 0.1) + 0.988771077936042*sin(theta)*cos(phi)*cos(psi + 0.1) - 0.149438132473599*cos(phi)*cos(theta))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1)))],
[                   21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi)*cos(phi))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(psi)*cos(theta)))],
[                   21*(0.988771077936042*sin(phi)*sin(psi) + 0.988771077936042*sin(theta)*cos(phi)*cos(psi) - 0.149438132473599*cos(phi)*cos(theta))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(psi)*cos(theta)))],
[ 21*(0.988771077936042*sin(phi)*sin(theta)*cos(psi - 0.1) - 0.149438132473599*sin(

In [28]:
gs.shape

(14, 1)

In [29]:
C = gs.jacobian([psi, theta, phi, w_x, w_y, w_z])

In [30]:
C

Matrix([
[  21*(-0.988771077936042*sin(phi)*sin(theta)*sin(psi + 0.1) - 0.988771077936042*cos(phi)*cos(psi + 0.1))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1))) + 2.65481066201837*(0.988771077936042*sin(phi)*sin(theta)*cos(psi + 0.1) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi + 0.1)*cos(phi))*sin(psi + 0.1)*cos(theta)/(0.151135218058295*sin(theta) + cos(theta)*cos(psi + 0.1))**2,    21*(0.149438132473599*sin(phi)*sin(theta) + 0.988771077936042*sin(phi)*cos(theta)*cos(psi + 0.1))/(8*(0.149438132473599*sin(theta) + 0.988771077936042*cos(theta)*cos(psi + 0.1))) + 2.68495986711101*(0.988771077936042*sin(theta)*cos(psi + 0.1) - 0.149438132473599*cos(theta))*(0.988771077936042*sin(phi)*sin(theta)*cos(psi + 0.1) - 0.149438132473599*sin(phi)*cos(theta) - 0.988771077936042*sin(psi + 0.1)*cos(phi))/(0.151135218058295*sin(theta) + cos(theta)*cos(psi + 0.1))**2,   21*(0.988771077936042*sin(phi)*sin(psi + 0.1) + 0.988771077936042*sin(theta

In [31]:
C.shape

(14, 6)

In [32]:
D = gs.jacobian([tau_1, tau_2, tau_3, tau_4])

In [33]:
D

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [34]:
D.shape

(14, 4)

#### C matrix is 14x6, x is 6x1 -> 14x6 x 6x1 = 14x1 = y. D = zeroes

we can now use LQR to find L matrix, similar to K matrix

convert C from sympy to numpy, object to float

In [35]:
C_eq = np.array(C.subs(phi, phie).subs(psi, psie).subs(theta, thetae), dtype=float)

In [36]:
C_eq

array([[-2.651426  ,  0.04000563, -0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [-0.04000563,  2.68556349,  0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        , -0.39672995,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.68495987,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.651426  , -0.04000563, -0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [ 0.04000563,  2.68556349, -0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  2.625     ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.651426  , -0.04000563,  0.3987219 ,  0.        ,  0.        ,
         0.        ],
       [ 0.04000563,  2.68556349,  0.26337851,  0.        ,  0.        ,
         0.        ],
       [-2.625     ,  0.        ,  0.39672995,  0.

In [37]:
C_eq.dtype

dtype('float64')

In [38]:
Q2 = np.eye(6)
R2 = np.eye(14)

L = lqr(A.T, C_eq.T, Q2, R2).T

In [39]:
L

array([[-4.30474997e-01, -6.49515535e-03, -4.26184577e-01,
         1.11728553e-17, -4.30474997e-01,  6.49515535e-03,
        -4.26184577e-01,  1.09233458e-17, -4.30474997e-01,
         6.49515535e-03, -4.26184577e-01,  1.11728553e-17,
        -4.30474997e-01, -6.49515535e-03],
       [ 6.39636391e-03,  4.29385591e-01,  3.82730943e-18,
         4.29289080e-01, -6.39636391e-03,  4.29385591e-01,
        -1.09233458e-17,  4.19702301e-01, -6.39636391e-03,
         4.29385591e-01, -2.56740011e-17,  4.29289080e-01,
         6.39636391e-03,  4.29385591e-01],
       [-6.02532678e-01,  3.98007140e-01, -5.99522524e-01,
        -9.98284038e-17, -6.02532678e-01, -3.98007140e-01,
        -4.84872561e-16, -9.75990603e-17,  6.02532678e-01,
         3.98007140e-01,  5.99522524e-01, -9.98284038e-17,
         6.02532678e-01, -3.98007140e-01],
       [-3.59781870e-01,  2.37656410e-01, -3.57984459e-01,
        -1.91867698e-16, -3.59781870e-01, -2.37656410e-01,
        -4.50681758e-16, -1.87582955e-16,  3.

In [40]:
Lm = sym.Matrix(L)

In [41]:
Lm

Matrix([
[ -0.430474996774142, -0.00649515534889632,   -0.426184576854397,  1.11728552993621e-17,   -0.430474996774142, 0.00649515534889634,    -0.426184576854397,  1.09233458272816e-17,   -0.430474996774142, 0.00649515534889644,    -0.426184576854397,  1.11728552993621e-17,  -0.430474996774142, -0.00649515534889642],
[0.00639636391175674,    0.429385590579361, 3.82730942806707e-18,     0.429289080032672, -0.00639636391175673,   0.429385590579361, -1.09233458272816e-17,     0.419702301285523, -0.00639636391175676,   0.429385590579361, -2.56740010826302e-17,     0.429289080032672, 0.00639636391175671,    0.429385590579361],
[ -0.602532677852903,     0.39800713985946,   -0.599522524179761, -9.98284037629455e-17,   -0.602532677852903,  -0.398007139859461, -4.84872560641696e-16, -9.75990602644257e-17,    0.602532677852902,    0.39800713985946,      0.59952252417976, -9.98284037629455e-17,   0.602532677852902,   -0.398007139859461],
[ -0.359781870240725,    0.237656410035834,    -0.35798445

In [42]:
L.shape

(6, 14)

In [43]:
np.linalg.eigvals(A-L@C_eq)

array([-6.91210085+0.j        , -1.01063246+0.j        ,
       -0.92798754+0.49706281j, -0.92798754-0.49706281j,
       -7.01022677+0.j        , -1.01033229+0.j        ])

## A, B, C, K, L

In [47]:
A, B, C_eq, K, L

(array([[ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1., -0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -0., -0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-3.74659401e-02,  7.49318801e-02, -3.74659401e-02,
          1.37647631e-17],
        [-6.48929117e-02, -9.17650871e-18,  6.48929117e-02,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.22171946e-02]]),
 array([[-2.651426  ,  0.04000563, -0.3987219 ,  0.        ,  0.        ,
          0.        ],
        [-0.04000563,  2.68556349,  0.26337851,  0.        ,  0.        ,
          0.        ],
        [-2.625     ,  0.        , -0.39

## -BK

In [48]:
-B@K

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.26394395e-17, -1.14491749e-16, -9.17724359e-02,
        -4.38140447e-01, -8.32667268e-17,  2.82380390e-16],
       [ 5.17002430e-17, -9.17724359e-02, -4.16333634e-17,
        -1.11022302e-16, -4.38140447e-01,  1.36130755e-16],
       [-6.22171946e-02,  6.23872516e-18,  1.61622217e-17,
         1.21657933e-16,  6.25680447e-17, -3.58197388e-01]])

## LC

In [51]:
L@C_eq

array([[ 7.92273330e+00,  2.01227923e-16,  1.30537942e-16,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.08166817e-16,  8.02055906e+00, -4.16333634e-17,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.23299762e-15, -1.55431223e-15,  1.85597508e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.58861593e-15, -3.77475828e-15,  1.10823232e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-7.80625564e-16,  7.08265845e+00, -4.16333634e-17,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.98559347e+00, -1.69829428e-15,  3.85975973e-16,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [52]:
A-B@K-L@C_eq

array([[-7.92273330e+00, -2.01227923e-16, -1.30537942e-16,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-2.08166817e-16, -8.02055906e+00,  4.16333634e-17,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00],
       [-8.23299762e-15,  1.55431223e-15, -1.85597508e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.51597649e-15,  3.66026653e-15, -1.20000475e+00,
        -4.38140447e-01, -8.32667268e-17,  2.82380390e-16],
       [ 8.32325807e-16, -7.17443089e+00,  0.00000000e+00,
        -1.11022302e-16, -4.38140447e-01,  1.36130755e-16],
       [-7.04781066e+00,  1.70453301e-15, -3.69813752e-16,
         1.21657933e-16,  6.25680447e-17, -3.58197388e-01]])

## F = A, -BK, LC, A-BK-LC

In [54]:
F = np.block([[A, -B@K],[L@C_eq, A-B@K-L@C_eq]])

In [56]:
f_eigvals = np.linalg.eigvals(F)

In [58]:
f_eigvals

array([-6.91210085+0.j        , -7.01022677+0.j        ,
       -0.17909869+0.17361121j, -0.17909869-0.17361121j,
       -0.92798754+0.49706281j, -0.92798754-0.49706281j,
       -1.01063246+0.j        , -1.01033229+0.j        ,
       -0.21907022+0.20923832j, -0.21907022-0.20923832j,
       -0.21907022+0.20923832j, -0.21907022-0.20923832j])

In [59]:
all(i < 0 for i in f_eigvals)

True